# BPO

In [ ]:
from koselleck import *

In [ ]:
def periodize(y,ybin=5):
    y1=y//ybin*ybin
    y2=y1+ybin
    return f'{y1}-{y2}'

In [ ]:
dfskip=get_skipgrams(calc_numlines=True)
dfskip['period']=dfskip.year.apply(periodize)
dfskip=dfskip.query('1700<=year<1900')
dfskip

In [ ]:
# dfgg=dfskip.groupby('period').sum()
# dfgg

In [ ]:
# dfskip[dfskip.num_lines<20000]

In [49]:
dfskipruns=pd.concat([
    dfskip.assign(run=f'run_{str(i+1).zfill(2)}')
    for i in range(1)
])
dfskipruns['opath']=dfskipruns.apply(lambda row: os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run,'model.bin'),1)
dfskipruns['opath_exists']=dfskipruns.opath.apply(lambda x: os.path.exists(x))
# dfskipruns=dfskipruns.query('run!="run_01"')
# dfskipruns=dfskipruns[dfskipruns.opath_exists==False]
dfskipruns

,corpus,year,path,num_lines,num_words,period,run,opath,opath_exists
157,bpo,1700,/home/ryan/github/koselleck/data/skipgrams/yea...,2232,55800,1700-1705,run_01,/home/ryan/github/koselleck/data/models/bpo/17...,True
42,bpo,1701,/home/ryan/github/koselleck/data/skipgrams/yea...,2475,61875,1700-1705,run_01,/home/ryan/github/koselleck/data/models/bpo/17...,True
25,bpo,1702,/home/ryan/github/koselleck/data/skipgrams/yea...,3001,75025,1700-1705,run_01,/home/ryan/github/koselleck/data/models/bpo/17...,True
288,bpo,1703,/home/ryan/github/koselleck/data/skipgrams/yea...,6546,163650,1700-1705,run_01,/home/ryan/github/koselleck/data/models/bpo/17...,True
69,bpo,1704,/home/ryan/github/koselleck/data/skipgrams/yea...,10910,272750,1700-1705,run_01,/home/ryan/github/koselleck/data/models/bpo/17...,True
...,...,...,...,...,...,...,...,...,...
21,bpo,1895,/home/ryan/github/koselleck/data/skipgrams/yea...,504743,12618575,1895-1900,run_01,/home/ryan/github/koselleck/data/models/bpo/18...,False
108,bpo,1896,/home/ryan/github/koselleck/data/skipgrams/yea...,490371,12259275,1895-1900,run_01,/home/ryan/github/koselleck/data/models/bpo/18...,False
17,bpo,1897,/home/ryan/github/koselleck/data/skipgrams/yea...,486894,12172350,1895-1900,run_01,/home/ryan/github/koselleck/data/models/bpo/18...,False
285,bpo,1898,/home/ryan/github/koselleck/data/skipgrams/yea...,495025,12375625,1895-1900,run_01,/home/ryan/github/koselleck/data/models/bpo/18...,False


In [50]:
def gen_and_save_model(dfskip,nskip=DEFAULT_NUM_SKIP,force=False,vector_size=100,window=10,min_count=5,epochs=10,workers=8):
    row=dfskip.iloc[0]
    odir=os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run)
    ofnfn=os.path.join(odir,'model.bin')
    if force or not os.path.exists(ofnfn):
        ensure_dir_exists(odir)
        ss=SkipgramsSamplers(dfskip.path, nskip)
        model = Word2Vec(sentences=ss,vector_size=vector_size,window=window,min_count=min_count,epochs=epochs,workers=workers)
        model.save(ofnfn)
    return pd.DataFrame()

# model.wv.most_similar('value',topn=25)

In [56]:
# enable_gensim_logging()
NSKIP_PER_YR=20000
# gen_and_save_model(dfskipruns.iloc[:5],force=True,nskip=NSKIP_PER_YR,epochs=10)
# m=load_model('/home/ryan/github/koselleck/data/models/bpo/1700-1705/run_01/model.bin')
# m.wv.most_similar('virtue')

In [55]:
# enable_gensim_logging()
disable_gensim_logging()

In [58]:
res=pmap_groups(
    gen_and_save_model,
    dfskipruns.groupby(['period','run']),
    num_proc=4,
    kwargs=dict(force=True, nskip=NSKIP_PER_YR)
)

Mapping gen_and_save_model [x4]: 100%|██████████| 40/40 [08:19<00:00, 12.48s/it]


In [60]:
m=load_model('/home/ryan/github/koselleck/data/models/bpo/1705-1710/run_01/model.bin')
m.wv.most_similar('virtue')

[('benefactors', 0.6232176423072815),
 ('vertue', 0.5589598417282104),
 ('unity', 0.5457756519317627),
 ('wisdom', 0.5376275181770325),
 ('innate', 0.5364933013916016),
 ('divine', 0.5251227617263794),
 ('consent', 0.5199053883552551),
 ('goodness', 0.5103564262390137),
 ('unalterable', 0.4965788722038269),
 ('eternal', 0.49366509914398193)]